# Generate Config (2433 patches)

In [1]:
mode = "sample" # "pastis"

In [2]:
import json
from datetime import datetime
from pathlib import Path
import hashlib

In [3]:
with open(f"../data/metadata_{mode}.geojson", "r") as f:
    geojson = json.load(f)

def compute_tar_digest(filepath, chunk_size=8192):
    h = hashlib.sha256()
    with open(filepath, 'rb') as f:
        while chunk := f.read(chunk_size):
            h.update(chunk)
    return f"sha256:{h.hexdigest()}"

def extract_temporal_coverage(props):
    all_dates = []
    for key in ["dates-S1A", "dates-S1D", "dates-S2"]:
        dates = props.get(key, {}).values()
        all_dates.extend(dates)
    all_dates = sorted(set(int(d) for d in all_dates))
    start = datetime.strptime(str(all_dates[0]), "%Y%m%d").strftime("%Y-%m-%d")
    end = datetime.strptime(str(all_dates[-1]), "%Y%m%d").strftime("%Y-%m-%d")
    return {"start": start, "end": end}

In [4]:
def compute_tar_digest(path):
    hash_fn = hashlib.sha256()
    with open(path, "rb") as f:
        while chunk := f.read(8192):
            hash_fn.update(chunk)
    return f"sha256:{hash_fn.hexdigest()}"

tar_dir = Path("../data/pastis")
layers = []

for tile_name in sorted(tile_metadata.keys()):
    tar_path = tar_dir / f"{tile_name}.tar"
    if not tar_path.exists():
        print(f"⚠️  Skipping {tile_name}: tar file not found at {tar_path}")
        continue

    print(f"\nProcessing {tar_path.name}...")
    digest = compute_tar_digest(tar_path)
    meta = tile_metadata[tile_name]

    layer = {
        "tile": tile_name,
        "bounding_box": meta.get("bounding_box"),
        "temporal_coverage": meta.get("dates"),
        "layer_digest": digest,
        "layer_mediaType": "application/vnd.oci.image.layer.v1.tar"
    }
    layers.append(layer)

config = {
    "schemaVersion": 2,
    "mediaType": "application/vnd.oci.image.config.v1+json",
    "artifactType": "application/vnd.whatever.v1+tar",
    "dataset": "PASTIS-HD",
    "created_by": "IGN",
    "license": "etalab-2.0",
    "tiles": layers
}

out_path = tar_dir / "config-2433.json"
with open(out_path, "w") as f:
    json.dump(config, f, indent=2)

Config written at /home/stefan/papers/oci-eo-data-packages/data/sample/config-2433.json)
